In [1]:
import time
import random
import math
import psutil
import os
import tracemalloc
import csv
from datetime import datetime

In [ ]:
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        merge_sort(left_half)
        merge_sort(right_half)

        i, j, k = 0, 0, 0
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

def selection_sort(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]

def block_sort(arr):
    n = len(arr)
    if n <= 1:
        return arr

    block_size = int(math.sqrt(n))
    blocks = [[] for _ in range(block_size)]

    max_value = max(arr)
    min_value = min(arr)
    range_value = max_value - min_value

    for num in arr:
        index = (num - min_value) * block_size // (range_value + 1)
        blocks[index].append(num)

    for block in blocks:
        block.sort()

    sorted_arr = []
    for block in blocks:
        sorted_arr.extend(block)
    
    return sorted_arr

def measure_time_and_memory(arr, sort_function):
    tracemalloc.start()
    start_time = time.time()
    sort_function(arr.copy())  # Ensure the original array is not altered
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return end_time - start_time, peak / (1024 * 1024)  # Convert to MB

def generate_best_case(n):
    return list(range(n))

def generate_worst_case(n):
    return list(range(n-1, -1, -1))

def generate_average_case(n):
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def read_input_file(filename):
    with open(filename, 'r') as file:
        sizes = list(map(int, file.readline().strip().split()))
        algorithm = file.readline().strip()
    return sizes, algorithm

def get_sort_function(algorithm_name):
    if algorithm_name == "MergeSort":
        return merge_sort
    elif algorithm_name == "QuickSort":
        return quick_sort
    elif algorithm_name == "SelectionSort":
        return selection_sort
    elif algorithm_name == "BlockSort":
        return block_sort
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm_name}")

def main():
    sizes, algorithm_name = read_input_file('input.txt')
    sort_function = get_sort_function(algorithm_name)
    log_file = 'sort_performance_log.csv'

    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    for size in sizes:
        cases = ['melhor', 'pior', 'medio']
        case_generators = [generate_best_case, generate_worst_case, generate_average_case]

        for case, gen_case in zip(cases, case_generators):
            arr = gen_case(size)
            print(f"Tamanho: {size}, Caso: {case}, Array: {arr}")
            
            if case == 'medio':
                times = []
                memories = []
                for _ in range(5):
                    arr = gen_case(size)
                    time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                    times.append(time_spent)
                    memories.append(memory_used)
                avg_time = sum(times) / len(times)
                avg_memory = sum(memories) / len(memories)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': avg_time,
                    'memory': avg_memory
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {avg_time:.6f}, {avg_memory:.6f} MB")
            else:
                time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': time_spent,
                    'memory': memory_used
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {time_spent:.6f}, {memory_used:.6f} MB")

            with open(log_file, 'a', newline='') as csvfile:
                fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(log_entry)

if __name__ == "__main__":
    main()

In [ ]:
if __name__ == "__main__":
    main()

Passos para Verificação Adicional
Prints Detalhados:

Adicione prints dentro da função block_sort para verificar o estado dos blocos após a distribuição e ordenação.
Verificação Manual:

Verifique manualmente com pequenas entradas para entender melhor o comportamento.

A simulação mostra que tanto no melhor quanto no pior caso, o merge_sort faz o mesmo número de divisões e mesclagens. A diferença de desempenho observada pode ser devida a outros fatores como otimizações internas do Python ou comportamento específico do ambiente de execução.

Se o problema persistir, pode ser útil realizar múltiplas execuções para coletar estatísticas mais robustas ou verificar se há outros fatores influenciando o desempenho.

In [2]:
import time
import random
import math
import tracemalloc
import os
import csv
from datetime import datetime

merge_operations = 0
quick_operations = 0
selection_operations = 0
block_operations = 0

def reset_counters():
    global merge_operations, quick_operations, selection_operations, block_operations
    merge_operations = 0
    quick_operations = 0
    selection_operations = 0
    block_operations = 0

def merge_sort(arr):
    global merge_operations
    
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        merge_sort(left_half)
        merge_sort(right_half)

        i, j, k = 0, 0, 0
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1
            
        merge_operations += 1  # Incrementa a contagem de mesclagens

def quick_sort(arr):
    global quick_operations
    
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]  # Usando o primeiro elemento como pivô
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        
        quick_operations += 1  # Incrementa a contagem de partições
        
        return quick_sort(left) + middle + quick_sort(right)

def selection_sort(arr):
    global selection_operations
    
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
            selection_operations += 1  # Incrementa a contagem de comparações
        arr[i], arr[min_idx] = arr[min_idx], arr[i]

def block_sort(arr):
    global block_operations
    
    n = len(arr)
    if n <= 1:
        return arr

    block_size = int(math.sqrt(n))
    blocks = [[] for _ in range(block_size)]

    max_value = max(arr)
    min_value = min(arr)
    range_value = max_value - min_value

    for num in arr:
        index = (num - min_value) * block_size // (range_value + 1)
        blocks[index].append(num)
        block_operations += 1  # Incrementa a contagem de operações de distribuição

    for block in blocks:
        block.sort()
        block_operations += 1  # Incrementa a contagem de operações de mesclagem

    sorted_arr = []
    for block in blocks:
        sorted_arr.extend(block)
    
    return sorted_arr

def measure_time_and_memory(arr, sort_function):
    tracemalloc.start()
    start_time = time.time()
    sort_function(arr.copy())  # Ensure the original array is not altered
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return end_time - start_time, peak / (1024 * 1024)  # Convert to MB

def generate_best_case(n):
    return list(range(n))

def generate_worst_case(n):
    return list(range(n-1, -1, -1))

def generate_average_case(n):
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def read_input_file(filename):
    with open(filename, 'r') as file:
        sizes = list(map(int, file.readline().strip().split()))
        algorithm = file.readline().strip()
    return sizes, algorithm

def get_sort_function(algorithm_name):
    if algorithm_name == "MergeSort":
        return merge_sort
    elif algorithm_name == "QuickSort":
        return quick_sort
    elif algorithm_name == "SelectionSort":
        return selection_sort
    elif algorithm_name == "BlockSort":
        return block_sort
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm_name}")

def main():
    global merge_operations, quick_operations, selection_operations, block_operations
    
    sizes, algorithm_name = read_input_file('input.txt')
    sort_function = get_sort_function(algorithm_name)
    log_file = 'sort_performance_log.csv'

    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory', 'operations']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    for size in sizes:
        cases = ['melhor', 'pior', 'medio']
        case_generators = [generate_best_case, generate_worst_case, generate_average_case]

        for case, gen_case in zip(cases, case_generators):
            arr = gen_case(size)
            print(f"Tamanho: {size}, Caso: {case}, Array: {arr}")
            
            reset_counters()  # Reset counters before each run
            
            if case == 'medio':
                times = []
                memories = []
                operations_list = []
                for _ in range(5):
                    arr = gen_case(size)
                    time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                    times.append(time_spent)
                    memories.append(memory_used)
                    operations_list.append(get_operations_count(algorithm_name))
                avg_time = sum(times) / len(times)
                avg_memory = sum(memories) / len(memories)
                avg_operations = sum(operations_list) / len(operations_list)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': avg_time,
                    'memory': avg_memory,
                    'operations': avg_operations
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {avg_time:.6f}, {avg_memory:.6f} MB, {avg_operations:.2f} operações")
            else:
                time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                operations = get_operations_count(algorithm_name)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': time_spent,
                    'memory': memory_used,
                    'operations': operations
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {time_spent:.6f}, {memory_used:.6f} MB, {operations:.2f} operações")

            with open(log_file, 'a', newline='') as csvfile:
                fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory', 'operations']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(log_entry)

def get_operations_count(algorithm_name):
    if algorithm_name == "MergeSort":
        return merge_operations
    elif algorithm_name == "QuickSort":
        return quick_operations
    elif algorithm_name == "SelectionSort":
        return selection_operations
    elif algorithm_name == "BlockSort":
        return block_operations
    else:
        return 0

if __name__ == "__main__":
    main()


Tamanho: 10, Caso: melhor, Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2024-07-04 11:24:36, SelectionSort, 10, melhor, 0.000076, 0.000214 MB, 45.00 operações
Tamanho: 10, Caso: pior, Array: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
2024-07-04 11:24:36, SelectionSort, 10, pior, 0.000062, 0.000214 MB, 45.00 operações
Tamanho: 10, Caso: medio, Array: [5, 2, 0, 3, 6, 4, 8, 9, 7, 1]
2024-07-04 11:24:36, SelectionSort, 10, medio, 0.000054, 0.000214 MB, 135.00 operações
Tamanho: 50, Caso: melhor, Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
2024-07-04 11:24:36, SelectionSort, 50, melhor, 0.002118, 0.000549 MB, 1225.00 operações
Tamanho: 50, Caso: pior, Array: [49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
2024-07-04 1

## imprimir comparações

In [5]:
import time
import random
import math
import tracemalloc
import os
import csv
from datetime import datetime

merge_operations = 0
quick_operations = 0
selection_operations = 0
block_operations = 0

def reset_counters():
    global merge_operations, quick_operations, selection_operations, block_operations
    merge_operations = 0
    quick_operations = 0
    selection_operations = 0
    block_operations = 0

def merge_sort(arr, depth=0):
    global merge_operations
    
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        print(f"{'  ' * depth}merge_sort(left_half): {left_half}")
        merge_sort(left_half, depth + 1)
        print(f"{'  ' * depth}merge_sort(right_half): {right_half}")
        merge_sort(right_half, depth + 1)

        i, j, k = 0, 0, 0
        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1
            
        merge_operations += 1  # Incrementa a contagem de mesclagens
        print(f"{'  ' * depth}Merged: {arr}")

def quick_sort(arr, depth=0):
    global quick_operations
    
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]  # Usando o primeiro elemento como pivô
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        
        quick_operations += 1  # Incrementa a contagem de partições
        print(f"{'  ' * depth}Pivot: {pivot}, Left: {left}, Middle: {middle}, Right: {right}")
        
        return quick_sort(left, depth + 1) + middle + quick_sort(right, depth + 1)

def selection_sort(arr):
    global selection_operations
    
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
            selection_operations += 1  # Incrementa a contagem de comparações
            print(f"Comparing: arr[{j}] ({arr[j]}) with arr[{min_idx}] ({arr[min_idx]}), min_idx: {min_idx}")
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
        print(f"Swapped: arr[{i}] ({arr[i]}) with arr[{min_idx}] ({arr[min_idx]}), Array: {arr}")

def block_sort(arr):
    global block_operations
    
    n = len(arr)
    if n <= 1:
        return arr

    block_size = int(math.sqrt(n))
    blocks = [[] for _ in range(block_size)]

    max_value = max(arr)
    min_value = min(arr)
    range_value = max_value - min_value

    for num in arr:
        index = (num - min_value) * block_size // (range_value + 1)
        blocks[index].append(num)
        block_operations += 1  # Incrementa a contagem de operações de distribuição
        print(f"Distributing: num ({num}) to block {index}, Blocks: {blocks}")

    for block in blocks:
        block.sort()
        block_operations += 1  # Incrementa a contagem de operações de mesclagem
        print(f"Sorted block: {block}, Blocks: {blocks}")

    sorted_arr = []
    for block in blocks:
        sorted_arr.extend(block)
    
    print(f"Final sorted array: {sorted_arr}")
    return sorted_arr

def measure_time_and_memory(arr, sort_function):
    tracemalloc.start()
    start_time = time.time()
    sort_function(arr.copy())  # Ensure the original array is not altered
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return end_time - start_time, peak / (1024 * 1024)  # Convert to MB

def generate_best_case(n):
    return list(range(n))

def generate_worst_case(n):
    return list(range(n-1, -1, -1))

def generate_average_case(n):
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def read_input_file(filename):
    with open(filename, 'r') as file:
        sizes = list(map(int, file.readline().strip().split()))
        algorithm = file.readline().strip()
    return sizes, algorithm

def get_sort_function(algorithm_name):
    if algorithm_name == "MergeSort":
        return merge_sort
    elif algorithm_name == "QuickSort":
        return quick_sort
    elif algorithm_name == "SelectionSort":
        return selection_sort
    elif algorithm_name == "BlockSort":
        return block_sort
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm_name}")

def main():
    global merge_operations, quick_operations, selection_operations, block_operations
    
    sizes, algorithm_name = read_input_file('input.txt')
    sort_function = get_sort_function(algorithm_name)
    log_file = 'sort_performance_log.csv'

    if not os.path.exists(log_file):
        with open(log_file, 'w', newline='') as csvfile:
            fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory', 'operations']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    for size in sizes:
        cases = ['melhor', 'pior', 'medio']
        case_generators = [generate_best_case, generate_worst_case, generate_average_case]

        for case, gen_case in zip(cases, case_generators):
            arr = gen_case(size)
            print(f"Tamanho: {size}, Caso: {case}, Array: {arr}")
            
            reset_counters()  # Reset counters before each run
            
            if case == 'medio':
                times = []
                memories = []
                operations_list = []
                for _ in range(5):
                    arr = gen_case(size)
                    time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                    times.append(time_spent)
                    memories.append(memory_used)
                    operations_list.append(get_operations_count(algorithm_name))
                avg_time = sum(times) / len(times)
                avg_memory = sum(memories) / len(memories)
                avg_operations = sum(operations_list) / len(operations_list)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': avg_time,
                    'memory': avg_memory,
                    'operations': avg_operations
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {avg_time:.6f}, {avg_memory:.6f} MB, {avg_operations:.2f} operações")
            else:
                time_spent, memory_used = measure_time_and_memory(arr, sort_function)
                operations = get_operations_count(algorithm_name)
                log_entry = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'algorithm': algorithm_name,
                    'size': size,
                    'case': case,
                    'time': time_spent,
                    'memory': memory_used,
                    'operations': operations
                }
                print(f"{log_entry['timestamp']}, {algorithm_name}, {size}, {case}, {time_spent:.6f}, {memory_used:.6f} MB, {operations:.2f} operações")

            with open(log_file, 'a', newline='') as csvfile:
                fieldnames = ['timestamp', 'algorithm', 'size', 'case', 'time', 'memory', 'operations']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(log_entry)

def get_operations_count(algorithm_name):
    if algorithm_name == "MergeSort":
        return merge_operations
    elif algorithm_name == "QuickSort":
        return quick_operations
    elif algorithm_name == "SelectionSort":
        return selection_operations
    elif algorithm_name == "BlockSort":
        return block_operations
    else:
        return 0

if __name__ == "__main__":
    main()


Tamanho: 10, Caso: melhor, Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Comparing: arr[1] (1) with arr[0] (0), min_idx: 0
Comparing: arr[2] (2) with arr[0] (0), min_idx: 0
Comparing: arr[3] (3) with arr[0] (0), min_idx: 0
Comparing: arr[4] (4) with arr[0] (0), min_idx: 0
Comparing: arr[5] (5) with arr[0] (0), min_idx: 0
Comparing: arr[6] (6) with arr[0] (0), min_idx: 0
Comparing: arr[7] (7) with arr[0] (0), min_idx: 0
Comparing: arr[8] (8) with arr[0] (0), min_idx: 0
Comparing: arr[9] (9) with arr[0] (0), min_idx: 0
Swapped: arr[0] (0) with arr[0] (0), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Comparing: arr[2] (2) with arr[1] (1), min_idx: 1
Comparing: arr[3] (3) with arr[1] (1), min_idx: 1
Comparing: arr[4] (4) with arr[1] (1), min_idx: 1
Comparing: arr[5] (5) with arr[1] (1), min_idx: 1
Comparing: arr[6] (6) with arr[1] (1), min_idx: 1
Comparing: arr[7] (7) with arr[1] (1), min_idx: 1
Comparing: arr[8] (8) with arr[1] (1), min_idx: 1
Comparing: arr[9] (9) with arr[1] (1), min_idx: 1
Swapped: a

Swapped: arr[25] (25) with arr[30] (27), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 48, 45, 31, 46, 27, 33, 36, 29, 37, 26, 35, 49, 30, 42, 39, 47, 41, 34, 40, 38, 32, 28, 43, 44]
Comparing: arr[27] (45) with arr[27] (45), min_idx: 27
Comparing: arr[28] (31) with arr[28] (31), min_idx: 28
Comparing: arr[29] (46) with arr[28] (31), min_idx: 28
Comparing: arr[30] (27) with arr[30] (27), min_idx: 30
Comparing: arr[31] (33) with arr[30] (27), min_idx: 30
Comparing: arr[32] (36) with arr[30] (27), min_idx: 30
Comparing: arr[33] (29) with arr[30] (27), min_idx: 30
Comparing: arr[34] (37) with arr[30] (27), min_idx: 30
Comparing: arr[35] (26) with arr[35] (26), min_idx: 35
Comparing: arr[36] (35) with arr[35] (26), min_idx: 35
Comparing: arr[37] (49) with arr[35] (26), min_idx: 35
Comparing: arr[38] (30) with arr[35] (26), min_idx: 35
Comparing: arr[39] (42) with arr[35] (26), min_idx: 35
Comparing: arr[40] (39) with arr[35] (26), min

Comparing: arr[83] (83) with arr[21] (21), min_idx: 21
Comparing: arr[84] (84) with arr[21] (21), min_idx: 21
Comparing: arr[85] (85) with arr[21] (21), min_idx: 21
Comparing: arr[86] (86) with arr[21] (21), min_idx: 21
Comparing: arr[87] (87) with arr[21] (21), min_idx: 21
Comparing: arr[88] (88) with arr[21] (21), min_idx: 21
Comparing: arr[89] (89) with arr[21] (21), min_idx: 21
Comparing: arr[90] (90) with arr[21] (21), min_idx: 21
Comparing: arr[91] (91) with arr[21] (21), min_idx: 21
Comparing: arr[92] (92) with arr[21] (21), min_idx: 21
Comparing: arr[93] (93) with arr[21] (21), min_idx: 21
Comparing: arr[94] (94) with arr[21] (21), min_idx: 21
Comparing: arr[95] (95) with arr[21] (21), min_idx: 21
Comparing: arr[96] (96) with arr[21] (21), min_idx: 21
Comparing: arr[97] (97) with arr[21] (21), min_idx: 21
Comparing: arr[98] (98) with arr[21] (21), min_idx: 21
Comparing: arr[99] (99) with arr[21] (21), min_idx: 21
Swapped: arr[21] (21) with arr[21] (21), Array: [0, 1, 2, 3, 4, 5

Comparing: arr[39] (60) with arr[39] (60), min_idx: 39
Comparing: arr[40] (59) with arr[40] (59), min_idx: 40
Comparing: arr[41] (58) with arr[41] (58), min_idx: 41
Comparing: arr[42] (57) with arr[42] (57), min_idx: 42
Comparing: arr[43] (56) with arr[43] (56), min_idx: 43
Comparing: arr[44] (55) with arr[44] (55), min_idx: 44
Comparing: arr[45] (54) with arr[45] (54), min_idx: 45
Comparing: arr[46] (53) with arr[46] (53), min_idx: 46
Comparing: arr[47] (52) with arr[47] (52), min_idx: 47
Comparing: arr[48] (51) with arr[48] (51), min_idx: 48
Comparing: arr[49] (50) with arr[49] (50), min_idx: 49
Comparing: arr[50] (49) with arr[50] (49), min_idx: 50
Comparing: arr[51] (48) with arr[51] (48), min_idx: 51
Comparing: arr[52] (47) with arr[52] (47), min_idx: 52
Comparing: arr[53] (46) with arr[53] (46), min_idx: 53
Comparing: arr[54] (45) with arr[54] (45), min_idx: 54
Comparing: arr[55] (44) with arr[55] (44), min_idx: 55
Comparing: arr[56] (43) with arr[56] (43), min_idx: 56
Comparing:

Swapped: arr[28] (28) with arr[85] (59), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 69, 36, 96, 81, 30, 41, 35, 54, 51, 43, 40, 38, 73, 98, 31, 60, 64, 93, 37, 79, 49, 67, 78, 33, 95, 92, 91, 94, 68, 61, 29, 80, 99, 74, 87, 66, 52, 85, 55, 34, 50, 97, 65, 82, 53, 58, 84, 90, 42, 46, 63, 39, 83, 89, 75, 71, 59, 76, 62, 88, 72, 48, 86, 57, 70, 44, 77, 56, 32, 45, 47]
Comparing: arr[30] (36) with arr[30] (36), min_idx: 30
Comparing: arr[31] (96) with arr[30] (36), min_idx: 30
Comparing: arr[32] (81) with arr[30] (36), min_idx: 30
Comparing: arr[33] (30) with arr[33] (30), min_idx: 33
Comparing: arr[34] (41) with arr[33] (30), min_idx: 33
Comparing: arr[35] (35) with arr[33] (30), min_idx: 33
Comparing: arr[36] (54) with arr[33] (30), min_idx: 33
Comparing: arr[37] (51) with arr[33] (30), min_idx: 33
Comparing: arr[38] (43) with arr[33] (30), min_idx: 33
Comparing: arr[39] (40) with arr[33] (30), min_idx: 33
Comparing: 

Swapped: arr[36] (36) with arr[52] (47), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 68, 76, 64, 98, 51, 63, 90, 48, 95, 55, 97, 84, 49, 47, 57, 37, 87, 41, 74, 78, 62, 61, 67, 82, 44, 99, 79, 50, 83, 69, 46, 45, 89, 54, 66, 71, 39, 86, 77, 75, 56, 81, 94, 93, 96, 91, 42, 53, 59, 85, 43, 58, 70, 72, 60, 80, 73, 52, 92, 65, 88]
Comparing: arr[38] (40) with arr[37] (38), min_idx: 37
Comparing: arr[39] (68) with arr[37] (38), min_idx: 37
Comparing: arr[40] (76) with arr[37] (38), min_idx: 37
Comparing: arr[41] (64) with arr[37] (38), min_idx: 37
Comparing: arr[42] (98) with arr[37] (38), min_idx: 37
Comparing: arr[43] (51) with arr[37] (38), min_idx: 37
Comparing: arr[44] (63) with arr[37] (38), min_idx: 37
Comparing: arr[45] (90) with arr[37] (38), min_idx: 37
Comparing: arr[46] (48) with arr[37] (38), min_idx: 37
Comparing: arr[47] (95) with arr[37] (38), min_idx: 37
Comparing: 

Comparing: arr[44] (62) with arr[44] (62), min_idx: 44
Comparing: arr[45] (68) with arr[44] (62), min_idx: 44
Comparing: arr[46] (85) with arr[44] (62), min_idx: 44
Comparing: arr[47] (60) with arr[47] (60), min_idx: 47
Comparing: arr[48] (49) with arr[48] (49), min_idx: 48
Comparing: arr[49] (84) with arr[48] (49), min_idx: 48
Comparing: arr[50] (64) with arr[48] (49), min_idx: 48
Comparing: arr[51] (77) with arr[48] (49), min_idx: 48
Comparing: arr[52] (50) with arr[48] (49), min_idx: 48
Comparing: arr[53] (72) with arr[48] (49), min_idx: 48
Comparing: arr[54] (74) with arr[48] (49), min_idx: 48
Comparing: arr[55] (66) with arr[48] (49), min_idx: 48
Comparing: arr[56] (61) with arr[48] (49), min_idx: 48
Comparing: arr[57] (59) with arr[48] (49), min_idx: 48
Comparing: arr[58] (76) with arr[48] (49), min_idx: 48
Comparing: arr[59] (45) with arr[59] (45), min_idx: 59
Comparing: arr[60] (47) with arr[59] (45), min_idx: 59
Comparing: arr[61] (86) with arr[59] (45), min_idx: 59
Comparing:

Comparing: arr[97] (93) with arr[77] (72), min_idx: 77
Comparing: arr[98] (81) with arr[77] (72), min_idx: 77
Comparing: arr[99] (80) with arr[77] (72), min_idx: 77
Swapped: arr[72] (72) with arr[77] (87), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 89, 85, 99, 90, 87, 77, 98, 88, 79, 97, 92, 86, 94, 82, 84, 73, 91, 74, 83, 78, 75, 76, 96, 95, 93, 81, 80]
Comparing: arr[74] (85) with arr[74] (85), min_idx: 74
Comparing: arr[75] (99) with arr[74] (85), min_idx: 74
Comparing: arr[76] (90) with arr[74] (85), min_idx: 74
Comparing: arr[77] (87) with arr[74] (85), min_idx: 74
Comparing: arr[78] (77) with arr[78] (77), min_idx: 78
Comparing: arr[79] (98) with arr[78] (77), min_idx: 78
Comparing: arr[80] (88) with arr[78] (77), min_idx: 78
Comparing: 

Swapped: arr[49] (49) with arr[95] (70), Array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 57, 63, 92, 51, 56, 97, 62, 82, 67, 50, 99, 55, 76, 81, 91, 60, 59, 78, 95, 88, 68, 94, 53, 58, 52, 80, 85, 54, 93, 86, 87, 65, 83, 71, 89, 66, 75, 61, 77, 74, 64, 79, 98, 73, 69, 70, 84, 96, 72, 90]
Comparing: arr[51] (63) with arr[50] (57), min_idx: 50
Comparing: arr[52] (92) with arr[50] (57), min_idx: 50
Comparing: arr[53] (51) with arr[53] (51), min_idx: 53
Comparing: arr[54] (56) with arr[53] (51), min_idx: 53
Comparing: arr[55] (97) with arr[53] (51), min_idx: 53
Comparing: arr[56] (62) with arr[53] (51), min_idx: 53
Comparing: arr[57] (82) with arr[53] (51), min_idx: 53
Comparing: arr[58] (67) with arr[53] (51), min_idx: 53
Comparing: arr[59] (50) with arr[59] (50), min_idx: 59
Comparing: arr[60] (99) with arr[59] (50), min_idx: 59
Comparing: 